In [1]:
import sys
sys.path.append('/home/samer/projects/fuzzy_sql/src') #This will enable reading the modules
from pathlib import Path
import os
import json

from fuzzy_sql.rnd_query import RND_QUERY
from fuzzy_sql.fuzzy_sql import *

In [2]:
#set directories
root_dir=Path('/home/samer/projects/fuzzy_sql')
metadata_dir=os.path.join(root_dir,'data/sdgd/processed/metadata')
db_path=os.path.join(root_dir,'db/sdgd.db')

In [3]:
#define input tables and metadata

tbl_names_lst=["C1"]

metadata_lst=[]
for tbl_name in tbl_names_lst:
    with open(os.path.join(metadata_dir,tbl_name+'.json'),'r') as f:
        metadata_lst.append(json.load(f))

In [4]:
#setup parameters
DFLT_PARAMS={
    'AGG_OPS':{'AVG':0.5, 'SUM':0.3, 'MAX':0.1, 'MIN':0.1 },
    'LOGIC_OPS':{'AND':0.9,'OR':0.1},
    'NOT_STATE':{'0':0.8, '1':0.2},
    'CAT_OPS':{'=':0.25, '<>':0.25, 'LIKE':0.15, 'IN':0.15, 'NOT LIKE':0.1, 'NOT IN':0.1},
    'CNT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0.1, '<=':0.1, '<>':0.1, 'BETWEEN':0.2, 'NOT BETWEEN':0.1},
    'DT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0, '<=':0, '<>':0.1, 'BETWEEN':0.2, 'IN':0.1, 'NOT BETWEEN':0.1, 'NOT IN':0.1},
    # 'FILTER_TYPE':{'WHERE':0.5, 'AND':0.5}, #Use WHERE or AND with JOIN CLAUSE
    # 'JOIN_TYPE': {'JOIN':0.5, 'LEFT JOIN':0.5}
}

#connect to db
conn = sqlite3.connect(db_path)

self=RND_QUERY(conn, tbl_names_lst, metadata_lst,DFLT_PARAMS)

Metadata fot table C1 validated.


# Generate Random  queries 

In [5]:
#Single Aggregate

rnd_query=self.make_single_agg_query(agg_fntn=True)


n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

SELECT marital_status, occupation, income, native_country, workclass, relationship, race, COUNT(*), AVG(fnlwgt)  FROM C1  GROUP BY marital_status, occupation, income, native_country, workclass, relationship, race
no_of_rows=5561
no_of_cols=9
product=50.049


SELECT marital_status, occupation, income, native_country, workclass, relationship, race, COUNT(*), AVG(fnlwgt)  FROM C1  GROUP BY marital_status, occupation, income, native_country, workclass, relationship, race


In [6]:
rnd_query['query_desc']

{'type': 'single_agg',
 'agg_fntn': 'None',
 'grpby_vars': ['marital_status',
  'occupation',
  'income',
  'native_country',
  'workclass',
  'relationship',
  'race'],
 'from_tbl_name': 'C1',
 'join_tbl_name_lst': [],
 'sql': 'SELECT marital_status, occupation, income, native_country, workclass, relationship, race, COUNT(*), AVG(fnlwgt)  FROM C1  GROUP BY marital_status, occupation, income, native_country, workclass, relationship, race',
 'n_rows': 5561,
 'n_cols': 9}

In [7]:
# Twin Aggregate 

syn_tbl_name_lst=['C1_syn_default_1']
rnd_query=self.make_twin_agg_query(syn_tbl_name_lst, agg_fntn=True)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(rnd_query['query_desc']['sql_real'])
print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")

print(rnd_query['query_desc']['sql_syn'])
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")



REAL:
no_of_rows=2543
no_of_cols=7
product=17.801


SYN:
no_of_rows=3594
no_of_cols=7
product=25.158


SELECT marital_status, occupation, race, sex, native_country, COUNT(*), AVG(age)  FROM C1  GROUP BY marital_status, occupation, race, sex, native_country


In [ ]:
rnd_query['query_desc']

In [ ]:
#Single Filter

# self.no_groupby_vars=2
# self.no_where_vars=2
# self.max_in_terms=2

rnd_query=self.make_single_fltr_query()


n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

In [ ]:
rnd_query['query_desc']

In [ ]:
# Twin Filter 


syn_tbl_name_lst=['C1_syn_default_1']
rnd_query=self.make_twin_fltr_query(syn_tbl_name_lst)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")
print(rnd_query['query_desc']['sql_real'])
print("\n")
print(rnd_query['query_desc']['sql_syn'])

In [ ]:
rnd_query['query_desc']